# Homework 1 - Berkeley STAT 157

Handout 1/22/2017, due 1/29/2017 by 4pm in Git by committing to your repository. Please ensure that you add the TA Git account to your repository.

1. Write all code in the notebook.
1. Write all text in the notebook. You can use MathJax to insert math or generic Markdown to insert figures (it's unlikely you'll need the latter). 
1. **Execute** the notebook and **save** the results.
1. To be safe, print the notebook as PDF and add it to the repository, too. Your repository should contain two files: ``homework1.ipynb`` and ``homework1.pdf``. 

The TA will return the corrected and annotated homework back to you via Git (please give `rythei` access to your repository).

In [ ]:
import numpy as np
import torch
import time


## 1. Speedtest for vectorization

Your goal is to measure the speed of linear algebra operations for different levels of vectorization. You need to use `wait_to_read()` on the output to ensure that the result is computed completely, since NDArray uses asynchronous computation. Please see http://beta.mxnet.io/api/ndarray/_autogen/mxnet.ndarray.NDArray.wait_to_read.html for details. 

1. Construct two matrices $A$ and $B$ with Gaussian random entries of size $4096 \times 4096$. 
1. Compute $C = A B$ using matrix-matrix operations and report the time. 
1. Compute $C = A B$, treating $A$ as a matrix but computing the result for each column of $B$ one at a time. Report the time.
1. Compute $C = A B$, treating $A$ and $B$ as collections of vectors. Report the time.
1. Bonus question - what changes if you execute this on a GPU?

In [ ]:
A=torch.randn(4096, 4096)
B=torch.randn(4096, 4096)

start_time=time.time()
C1=torch.mm(A,B)
print(time.time()-start_time)


1.6290855407714844


In [ ]:
C2=torch.zeros_like(A)
start_time=time.time()
for i in range(4096):
  C2[:,i]=torch.mv(A,B[:,i])
print(time.time()-start_time)

21.87451720237732


In [ ]:
C3=torch.zeros_like(A)
start_time=time.time()
for i in range(4096):
  for j in range(4096):
    C3[i,i]=torch.dot(A[i,:],B[:,j])
print(time.time()-start_time)

## 2. Semidefinite Matrices

Assume that $A \in \mathbb{R}^{m \times n}$ is an arbitrary matrix and that $D \in \mathbb{R}^{n \times n}$ is a diagonal matrix with nonnegative entries. 

1. Prove that $B = A D A^\top$ is a positive semidefinite matrix. 
1. When would it be useful to work with $B$ and when is it better to use $A$ and $D$?

## 3. MXNet on GPUs

1. Install GPU drivers (if needed)
1. Install MXNet on a GPU instance
1. Display `!nvidia-smi`
1. Create a $2 \times 2$ matrix on the GPU and print it. See http://d2l.ai/chapter_deep-learning-computation/use-gpu.html for details.

In [ ]:
!nvidia-smi

Mon Jun 21 06:42:16 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
X = torch.ones(2, 3, device='cuda:0')
X
torch.cuda.device_count()

1

## 4. NDArray and NumPy 

Your goal is to measure the speed penalty between MXNet Gluon and Python when converting data between both. We are going to do this as follows:

1. Create two Gaussian random matrices $A, B$ of size $4096 \times 4096$ in NDArray. 
1. Compute a vector $\mathbf{c} \in \mathbb{R}^{4096}$ where $c_i = \|A B_{i\cdot}\|^2$ where $\mathbf{c}$ is a **NumPy** vector.

To see the difference in speed due to Python perform the following two experiments and measure the time:

1. Compute $\|A B_{i\cdot}\|^2$ one at a time and assign its outcome to $\mathbf{c}_i$ directly.
1. Use an intermediate storage vector $\mathbf{d}$ in NDArray for assignments and copy to NumPy at the end.

In [ ]:

A=torch.randn(4096, 4096)
B=torch.randn(4096, 4096)

c1=np.zeros(4096)
start_time=time.time()
for i in range(4096):
    c1[i]=(torch.norm(torch.mv(A,B[:,i])))**2
print(time.time()-start_time)

21.715687036514282


In [ ]:
d=torch.zeros(4096)
start_time=time.time()
for i in range(4096):
    d[i]=(torch.norm(torch.mv(A,B[:,i])))**2
c2=d.numpy
print(time.time()-start_time)

21.706103324890137


## 5. Memory efficient computation

We want to compute $C \leftarrow A \cdot B + C$, where $A, B$ and $C$ are all matrices. Implement this in the most memory efficient manner. Pay attention to the following two things:

1. Do not allocate new memory for the new value of $C$.
1. Do not allocate new memory for intermediate results if possible.

In [ ]:

A=torch.randn(100, 100)
B=torch.randn(100, 100)
C=torch.randn(100, 100)

before=id(C)
C+=torch.mm(A,B)
id(C)==before
C

tensor([[ -8.6098,   2.4111,   9.8159,  ...,   2.9815,   8.0476, -12.3791],
        [ -4.1424,  -9.8955,   9.0422,  ...,  -6.2098,  10.5725,   1.8603],
        [  5.5484,  -4.0713,  -3.2330,  ...,  -2.4263,   1.5679,  -3.2859],
        ...,
        [-11.2820,  16.7010,   5.1281,  ...,  -7.5853,  13.2953, -27.5719],
        [ 13.7232,  -1.3100,  -0.2509,  ...,   1.3982, -23.2251,   6.4915],
        [-10.6417,   3.1863,  11.4934,  ...,  -8.0808,   8.7744, -13.8706]])

## 6. Broadcast Operations

In order to perform polynomial fitting we want to compute a design matrix $A$ with 

$$A_{ij} = x_i^j$$

Our goal is to implement this **without a single for loop** entirely using vectorization and broadcast. Here $1 \leq j \leq 20$ and $x = \{-10, -9.9, \ldots 10\}$. Implement code that generates such a matrix.

In [ ]:
import torch
x=torch.arange(-10,10.1,0.1).reshape(201,1)
j=torch.arange(1,21).reshape(1,20)
print(x**j)


RuntimeError: ignored